# Importar librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, confusion_matrix, accuracy_score, jaccard_score, f1_score, roc_curve, auc
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, LogisticRegression

# Armar un dataframe

In [ ]:
df = pd.read_csv('weatherAUS.csv')
df_subset = df.iloc[:,1:]

# Vistazo rapido del dataframe

In [ ]:
df_subset.head()



In [ ]:
df_subset.tail(5)

In [ ]:
df_subset.describe()

In [ ]:
df_subset.info()

# Ver las columnas e identificar cada una

In [ ]:
df_subset.columns

25 Columnas

**Unnamed** Indices (no esta en df_subset)

**Date** fecha de la observacion

**Location** Nombre del lugar donde esta la estacion meteorologica

**MinTemp** Temperatura minima. En grados celsius

**MaxTemp** Temperatura maxima. En grados celsius

**Rainfall** cantidad de lluvia registrada en el dia. En mm

**Evaporation** evaporacion (mm) de 00 a 09am.

**Sunshine** Numero de horas de luz solar durante el dia.

**WindGustDir** direccion de la rafaga de viento mas fuerte en las 24 horas

**WindGustSpeed** velocidad de la rafaga de viento mas fuerte en km/h

**WindDir9am** direccion del viento a las 9 am

**WindDir3pm** direccion del viento a las 3 pm

**WindSpeed9am** velocidad del viento en km/h, a las 9 am

**WindSpeed3pm** velocidad del viento en km/h, a las 3 pm

**Humidity9am** humedad en porcentaje a las 9 am

**Humidity3pm** humedad en porcentaje a las 3 pm

**Pressure9am** presion atmosferica en (hpa) al nivel del mar a las 9 am

**Pressure9am** presion atmosferica en (hpa) al nivel del mar a las 3 pm

**Cloud9am** Fraccion del cielo oscurecida por nubes medida en fracciones de 8 (0 indica sin nubes, 8 totalmente nublado) a las 9 am

**Cloud9am** Fraccion del cielo oscurecida por nubes medida en fracciones de 8 (0 indica sin nubes, 8 totalmente nublado) a las 3pm

**Temp9am** temperatura en grados celsius a las 9 am

**Temp3pm** temperatura en grados celsius a las 3 pm

**RainToday** valor booleano si llovio o no durante el dia (1 si pasa 1 mm)

**RainTomorrow** Cantidad de lluvia al dia siguiente en mm

**RainfallTomorrow** cantidad de lluvia al dia siguiente en mm

# Que no voy a usar
Date No se va a usar
Localization sera descartada


In [ ]:
df_subset = df_subset.drop(['Date'], axis=1)

# codear localizacion para costa este solamente

In [ ]:
locations_replace= ["Sydney", "SydneyAirport", "Canberra","Melbourne", "MelbourneAirport"]
df_subset['Location'] = df_subset['Location'].replace(locations_replace, 'East Coast')


filtered_df = df_subset[df_subset['Location'] == 'East Coast']


filtered_df = filtered_df.reset_index(drop=True)

In [ ]:
filtered_df = filtered_df.drop(['Location'], axis=1)
filtered_df.head(10)

# Que valores son numericos y cuales son categoricos

In [ ]:
def num_or_cat(df):
    categorical = df.select_dtypes(include='object').columns
    numerical = df.select_dtypes(exclude='object').columns

    return(categorical, numerical)

In [ ]:
categorical_cols, numerical_cols= num_or_cat(filtered_df)


In [ ]:
categorical_cols_graph = categorical_cols.delete(0)
categorical_cols

In [ ]:
numerical_cols

In [ ]:
for i in range(len(numerical_cols)):
    plt.figure(figsize=(3,3))
    sns.histplot(filtered_df[numerical_cols[i]], color="blue")
    label=numerical_cols[i]
    plt.xlabel(numerical_cols[i])
    plt.ylabel("count")
    plt.title(label)

# Contando valores faltantes

In [ ]:
filtered_df.isnull().sum()

# llenar columnas numericas

In [ ]:
imputerResto = KNNImputer(missing_values=np.nan, n_neighbors=2)

for columna in numerical_cols:
   filtered_df[columna] = imputerResto.fit_transform(filtered_df[[columna]])

In [ ]:
filtered_df.isnull().sum()

Capaz saque rainfall tomorrow del loop

# Cambiar Datos categoricos

En el caso de las columnas RainToday y RainTomorrow pasar Yes a 1 y no a 0

In [ ]:
categorical_cols

In [ ]:
filtered_df['RainToday']=filtered_df['RainToday'].fillna('No')
filtered_df['RainTomorrow']=filtered_df['RainTomorrow'].fillna('No')


# El resto de columnas categoricas reemplazar valores faltantes por la moda

In [ ]:
filtered_df['WindGustDir'] = filtered_df['WindGustDir'].fillna(filtered_df['WindGustDir'].mode()[0])
filtered_df['WindDir9am'] = filtered_df['WindDir9am'].fillna(filtered_df['WindDir9am'].mode()[0])
filtered_df['WindDir3pm'] = filtered_df['WindDir3pm'].fillna(filtered_df['WindDir3pm'].mode()[0])

# ----------------------------------------------------------------------------------------------------

# Graficar datos numericos

In [ ]:
for i in range(len(numerical_cols)):
    plt.figure(figsize=(3,3))
    sns.histplot(filtered_df[numerical_cols[i]], color="blue")
    label=numerical_cols[i]
    plt.xlabel(numerical_cols[i])
    plt.ylabel("count")
    plt.title(label)



# Graficar Categoricos


In [ ]:
for i in range(len(categorical_cols_graph)):
    plt.figure(figsize=(5,5))
    sns.histplot(filtered_df[categorical_cols_graph[i]], color="blue")
    label=categorical_cols_graph[i]
    plt.xlabel(categorical_cols_graph[i])
    plt.ylabel("count")
    plt.title(label)

In [ ]:
filtered_df.isnull().sum()

# Check si esta balanceado

no esta balanceado (por mas que ya haya reemplazado lso valores faltantes sigue estando desbalanceado)

In [ ]:
plt.figure(figsize=(3,3))
sns.countplot(x=df_subset['RainTomorrow'])
plt.title("Balanceado")
plt.show()

# Preparar dataset

In [ ]:
copia_df = filtered_df.copy()

# Codear los datos categoricos

In [ ]:
le = LabelEncoder()
for columna in categorical_cols:
    copia_df[columna] = le.fit_transform(copia_df[columna])

# Hacer matriz de correlacion

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(copia_df.corr(),annot=True, cmap="Blues")
plt.show()

# Dividir en train test

In [ ]:
copia_df.head()

In [ ]:
X = copia_df.iloc[:,:-2].values
X

In [ ]:
y = copia_df['RainfallTomorrow'].values

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
X_test

# Crear los modelos de regression lasso ridge elasticnet y gradiente

In [ ]:
lasso = Lasso(alpha=0.1)
ridge = Ridge(alpha=0.1)  
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.5)
grad = SGDRegressor(max_iter=1000)

# Normalizar los datos

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
grad.fit(X_train_scaled, y_train)

# Predecir

In [ ]:
y_pred = grad.predict(X_test_scaled)


# Pruebas

In [ ]:
r_squared = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
print("R2:", r_squared)
print("Mean Abs Precentage error:", mape)
print("Mean Sq Error", mse)
print("Mean Abs Error", mae)

# Regresion Logistica usando (Rain tomorrow)

In [ ]:
copia_df_log = copia_df.drop(['RainfallTomorrow'], axis=1)

In [ ]:
X_log = copia_df_log.iloc[:,:-1].values
y_log = copia_df_log['RainTomorrow']
X_train_log,X_test_log,y_train_log,y_test_log = train_test_split(X_log,y_log,test_size=0.25,random_state=1)


# Normalizar

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_log=MinMaxScaler()
X_train_log=scaler_log.fit_transform(X_train_log)
X_test_log=scaler_log.transform(X_test_log)

# Crear modelo de reg logistica

In [ ]:
reg_log = LogisticRegression(random_state = 0)
reg_log.fit(X_train_log, y_train_log)

In [ ]:
predictions = reg_log.predict(X_test_log)

In [ ]:
cm = confusion_matrix(y_test_log, predictions)

LR_Accuracy_Score = accuracy_score(y_test_log, predictions)


LR_JaccardIndex = jaccard_score(y_test_log, predictions, average='weighted')


LR_F1_Score = f1_score(y_test_log, predictions, average='weighted')




print(cm)
print("reg_log Accuracy:", LR_Accuracy_Score)
print("reg_log Jaccard Index:", LR_JaccardIndex)
print("reg_log F1:", LR_F1_Score)


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_log, predictions)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlabel('Valores False Positive ')
plt.ylabel('Valores True positive')
plt.title('ROC')
plt.show()

# Ejercicio 5 crear modelos base de regresion y clasificacion

Primero regresion

In [ ]:
copia_df.head()

In [ ]:
X_base_reg = copia_df[['Rainfall','Humidity3pm']].values
y_base_reg = copia_df[['RainfallTomorrow']].values
#######         ##########
X_train_base_reg, X_test_base_reg, y_train_base_reg, y_test_base_reg = train_test_split(X_base_reg, y_base_reg, test_size=0.1)
#######         ##########
X_base_clas = copia_df[['Cloud3pm','WindDir9am']]
y_base_clas = copia_df[['RainTomorrow']]
#######         ##########
X_train_base_clas, X_test_base_clas, y_train_base_clas, y_test_base_clas = train_test_split(X_base_clas, y_base_clas, test_size=0.1)

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train_base_reg, y_train_base_reg)


In [ ]:
y_pred_base_reg = regressor.predict(X_test_base_reg)

r_cuad = r2_score(y_test_base_reg, y_pred_base_reg)
print(f"R^2 score: {r_cuad}")

Clasificacion

In [ ]:
classificator = LogisticRegression()
classificator.fit(X_train_base_clas, y_train_base_clas)

In [ ]:
y_pred_base_clas = classificator.predict(X_test_base_clas)

In [ ]:
acc = accuracy_score(y_test_base_clas, y_pred_base_clas)
print(f"Precision del modelo de regresion logistica base: {acc}")

# Ejercicio 6 Entrenar una red neuronal

In [ ]:
import tensorflow as tf
import optuna
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense
from sklearn.datasets import load_iris

In [ ]:
X_ann = copia_df.iloc[:,:-1].values
y_ann = copia_df['RainTomorrow'].values
X_train_ann,X_test_ann,y_train_ann,y_test_ann = train_test_split(X_ann,y_ann,test_size=0.2,random_state=1)

In [ ]:
data = load_iris()
X_train, X_test_ann, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42) # necesitamos un conjunto de validación para obtener hiperparámetros.

In [ ]:
ann = tf.keras.models.Sequential()

In [71]:
def objective(trial):

    lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    num_layers = trial.suggest_int('num_layers', 1, 3)


    for i in range(num_layers):
        # Sugerir activaciones
        activation = trial.suggest_categorical(f'activation_layer_{i}', ['relu', 'sigmoid', 'tanh'])
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128)
        ann.add(Dense(num_units, activation=activation))

    # capa de salida
    ann.add(Dense(1, activation='sigmoid')) #la clase de salida

    # compilar
    ann.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # entrenar
    ann.fit(X_train_ann, y_train_ann, validation_data=(X_test_ann, y_test_ann), epochs=5, batch_size=32, verbose=0)

    # evaluar
    score = ann.evaluate(X_test_ann, y_test_ann, verbose=0)
    return score[1]

# crear un estudio de Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)

# obtener los mejores hiperparámetros
best_params = study.best_params
print("Best parámetros encontrados:", best_params)

[I 2023-11-16 20:52:56,537] A new study created in memory with name: no-name-8685465d-fab4-41bf-913d-8668fa346003
[I 2023-11-16 20:53:28,314] Trial 0 finished with value: 0.7714196443557739 and parameters: {'learning_rate': 0.005551347957030787, 'num_layers': 3, 'activation_layer_0': 'sigmoid', 'n_units_layer_0': 18, 'activation_layer_1': 'sigmoid', 'n_units_layer_1': 20, 'activation_layer_2': 'tanh', 'n_units_layer_2': 93}. Best is trial 0 with value: 0.7714196443557739.
[I 2023-11-16 20:54:06,535] Trial 1 finished with value: 0.7714196443557739 and parameters: {'learning_rate': 0.0036941394123863433, 'num_layers': 3, 'activation_layer_0': 'relu', 'n_units_layer_0': 93, 'activation_layer_1': 'tanh', 'n_units_layer_1': 39, 'activation_layer_2': 'tanh', 'n_units_layer_2': 122}. Best is trial 0 with value: 0.7714196443557739.


Best parámetros encontrados: {'learning_rate': 0.005551347957030787, 'num_layers': 3, 'activation_layer_0': 'sigmoid', 'n_units_layer_0': 18, 'activation_layer_1': 'sigmoid', 'n_units_layer_1': 20, 'activation_layer_2': 'tanh', 'n_units_layer_2': 93}
